# SDR Neural Network

This example of neural network is used to recognize patterns of a simple display, which can represent numbers from 0 to 9.
<img src=https://upload.wikimedia.org/wikipedia/commons/thumb/0/02/7_segment_display_labeled.svg/1200px-7_segment_display_labeled.svg.png width=200/>




In [3]:
import numpy as np

In [4]:
class Perceptron:
    """ A single neuron with the sigmoid activate function
        Attributtes:
            inputs: The number of inputs in the perceptron, not cunting the bias
            bias: the bias term. By default it´s 1.0
    """
    
    def __init__(self, inputs, bias = 1.0):
        """Return a new Perceptron object with the specified number of inputs +1 (for the bias)"""
        self.weights = np.random.rand(inputs+1)*2 -1
        self.bias = bias
        
    def run(self,x):
        """Run the perceptron. x is a python list with the input values."""
        x_sum = np.dot(np.append(x,self.bias),self.weights)
        #this calculates the product point of the inputs and the weights
        return self.sigmoid(x_sum)
    
    def set_weights(self,w_init):
        """Set the weights. w_init is a python list with the weights"""
        self.weights = np.array(w_init)

    
    def sigmoid(self,x):
        """Evaluate the sigmoid function for thw floating point input x"""
        return 1/(1 + np.exp(-x))

In [5]:

class MultiLayerPerceptron:
    """A multilayer perceptron class that uses the perceptron class above.
        Attributtes:
            layers: A python list with the number of elements per layer
            bias: The bias term. The same bias is used for all neurons
            eta: The learning rate"""
    def __init__(self,layers,bias = 1.0, eta = 0.5):
        """Return a new MLP object with the specified parameters"""
        self.layers = np.array(layers,dtype=object)
        self.bias = bias
        self.eta = eta
        self.network = [] #the list of all neurons
        self.values = [] #the list of all output values
        self.d= []
        
        for i in range(len(self.layers)):
            self.values.append([])
            self.d.append([])
            self.network.append([])
            self.values[i] = [0.0 for j in range(self.layers[i])]
            self.d[i] = [0.0 for j in range(self.layers[i])]
            if i>0:
                for j in range(self.layers[i]):
                    self.network[i].append(Perceptron(inputs=self.layers[i-1],bias=self.bias))
    
        self.network = np.array([np.array(x) for x in self.network],dtype=object)
        self.values=np.array([np.array(x) for x in self.values],dtype=object)
        self.d = np.array([np.array(x) for x in self.d],dtype=object)
    
    def set_weights(self,w_init):
        """set the weights.
            w_init is a list of lists with the weights for all, but the input layer"""
        for i in range(len(w_init)):
            for j in range(len(w_init[i])):
                self.network[i+1][j].set_weights(w_init[i][j])
                
    def printWeights(self):
        print()
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                print("Layer",i+1,"Neuron",j,self.network[i][j].weights)
            print()
            
    def run(self,x):
        """Feed a sample x into the multilayer perceptron"""
        x=np.array(x,dtype=object)
        self.values[0]=x
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                self.values[i][j]= self.network[i][j].run(self.values[i-1])
        return self.values[-1]
        
    #Backpropagation method
    def bp(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorythm)."""
        x = np.array(x, dtype=object)
        y = np.array(y, dtype=object)

        #Step 1:  feed a sample to the network
        outputs = self.run(x)

        #Step 2: calculate the MSE
        error = (y-outputs)
        MSE = sum(error ** 2)/ self.layers[-1]

        #Step 3: Calculate the output error terms
        self.d[-1] = outputs * (1-outputs)*error

        #Step 4: Calculate the error term of each unit on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]):
                    fwd_error+= self.network[i+1][k].weights[h] * self.d[i+1][k]
                self.d[i][h]=self.values[i][h] * (1-self.values[i][h]) * fwd_error

        #step 5 & 6 : calculate the deltas and update the weights
        #iterates layers
        for i in range(1, len(self.network)):
            #iterates neurons
            for j in range(self.layers[i]):
                #iterates inputs
                for k in range(self.layers[i-1]+1):
                    if k == self.layers[i-1]:
                        delta = self.eta * self.d[i][j] *self.bias
                    else:
                        delta = self.eta * self.d[i][j] * self.values[i-1][k]
                    self.network[i][j].weights[k]+=delta

        return MSE

In [ ]:
#test code
epochs = int(input("How many epochs?"))
mlp1 = MultiLayerPerceptron(layers=[7,7,1])
mlp2 = MultiLayerPerceptron(layers=[7,7,10])
mlp3 = MultiLayerPerceptron(layers=[7,7,7])

print("Training 7 to 1 network ...")
#dataset for the 7 to 1 network
for i in range(epochs):
    mse=0.0
    mse+=mlp1.bp([1,1,1,1,1,1,0],[0.05])
    mse+=mlp1.bp([0,1,1,0,0,0,0],[0.15])
    mse+=mlp1.bp([1,1,0,1,1,0,1],[0.25])
    mse+=mlp1.bp([1,1,1,1,0,0,1],[0.35])
    mse+=mlp1.bp([0,1,1,0,0,1,1],[0.45])
    mse+=mlp1.bp([1,0,1,1,0,1,1],[0.55])
    mse+=mlp1.bp([1,0,1,1,1,1,1],[0.65])
    mse+=mlp1.bp([1,1,1,0,0,0,0],[0.75])
    mse+=mlp1.bp([1,1,1,1,1,1,1],[0.85])
    mse+=mlp1.bp([1,1,1,1,0,1,1],[0.95])
    mse = mse/10

print("Training 7 to 10 network ...")
#Dataset for the 7 to 10 network
for i in range(epochs):
    mse=0.0
    mse+=mlp2.bp([1,1,1,1,1,1,0],[1,0,0,0,0,0,0,0,0,0])
    mse+=mlp2.bp([0,1,1,0,0,0,0],[0,1,0,0,0,0,0,0,0,0])
    mse+=mlp2.bp([1,1,0,1,1,0,1],[0,0,1,0,0,0,0,0,0,0])
    mse+=mlp2.bp([1,1,1,1,0,0,1],[0,0,0,1,0,0,0,0,0,0])
    mse+=mlp2.bp([0,1,1,0,0,1,1],[0,0,0,0,1,0,0,0,0,0])
    mse+=mlp2.bp([1,0,1,1,0,1,1],[0,0,0,0,0,1,0,0,0,0])
    mse+=mlp2.bp([1,0,1,1,1,1,1],[0,0,0,0,0,0,1,0,0,0])
    mse+=mlp2.bp([1,1,1,0,0,0,0],[0,0,0,0,0,0,0,1,0,0])
    mse+=mlp2.bp([1,1,1,1,1,1,1],[0,0,0,0,0,0,0,0,1,0])
    mse+=mlp2.bp([1,1,1,1,0,1,1],[0,0,0,0,0,0,0,0,0,1])
    mse = mse/10

print("Training 7 to 7 network ...")
#Dataset for 7 to 7 network
for i in range(epochs):
    mse=0.0
    mse+=mlp3.bp([1,1,1,1,1,1,0],[1,1,1,1,1,1,0])
    mse+=mlp3.bp([0,1,1,0,0,0,0],[0,1,1,0,0,0,0])
    mse+=mlp3.bp([1,1,0,1,1,0,1],[1,1,0,1,1,0,1])
    mse+=mlp3.bp([1,1,1,1,0,0,1],[1,1,1,1,0,0,1])
    mse+=mlp3.bp([0,1,1,0,0,1,1],[0,1,1,0,0,1,1])
    mse+=mlp3.bp([1,0,1,1,0,1,1],[1,0,1,1,0,1,1])
    mse+=mlp3.bp([1,0,1,1,1,1,1],[1,0,1,1,1,1,1])
    mse+=mlp3.bp([1,1,1,0,0,0,0],[1,1,1,0,0,0,0])
    mse+=mlp3.bp([1,1,1,1,1,1,1],[1,1,1,1,1,1,1])
    mse+=mlp3.bp([1,1,1,1,0,1,1],[1,1,1,1,0,1,1])
    mse = mse/10
    
pattern = [1.2]
while(pattern[0]>=0.0):
    pattern = list(map(float,input("Input pattern 'a b c d e f g':").strip().split()))
    if(pattern[0]<0.0):
        break
    print()
    print("The number recognized by the 7 to 1 network is: ", int(mlp1.run(pattern)*10))
    print("The number recognized by the 7 to 10 network is: ", mlp2.run(pattern))
    print("The number recognized by the 7 to 7 network is: ", [int(x) for x in (mlp3.run(pattern)+0.5)],"\n")



How many epochs?1000
Training 7 to 1 network ...
Training 7 to 10 network ...
Training 7 to 7 network ...
Input pattern 'a b c d e f g':1 1 1 1 0 1 1 

The number recognized by the 7 to 1 network is:  8
The number recognized by the 7 to 10 network is:  [2.83478721e-02 3.52516962e-04 5.11598584e-04 7.86678401e-02
 4.52836389e-02 4.39168678e-02 2.82695967e-04 3.23383723e-02
 4.19715445e-02 9.20259835e-01]
The number recognized by the 7 to 7 network is:  [1, 1, 1, 1, 0, 1, 1] 

Input pattern 'a b c d e f g':0 1 1 0 0 1 1

The number recognized by the 7 to 1 network is:  4
The number recognized by the 7 to 10 network is:  [2.54716567e-03 5.76089663e-02 4.68548305e-05 2.78645696e-03
 9.37305631e-01 1.47071991e-02 1.26531558e-03 1.95673110e-03
 2.90204432e-02 2.53196320e-02]
The number recognized by the 7 to 7 network is:  [0, 1, 1, 0, 0, 1, 1] 

